In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from commando import ComManDo
from commando.evaluation import *
from commando.utilities import *
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

2022-10-03 06:03:06.556892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 06:03:06.556939: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
np.random.seed(42)

In [4]:
import scanpy as sc

babel_dir = '../babel_data/DM_rep4/'
train = sc.read_h5ad(babel_dir + 'train_rna.h5ad')
v = train.var_names
train = train.X.toarray()
valid = sc.read_h5ad(babel_dir + 'truth_rna.h5ad').X.toarray()
data1 = np.concatenate([train, valid], axis=0)
fnames1 = np.array(v)

train = sc.read_h5ad(babel_dir + 'train_atac.h5ad')
v = train.var_names
train = train.X.toarray()
valid = sc.read_h5ad(babel_dir + 'truth_atac.h5ad').X.toarray()
data2 = np.concatenate([train, valid], axis=0)
fnames2 = np.array(v)

split = train.shape[0]

/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  warn(
/home/thema/miniconda3/lib/python3.9/site-packages/anndata/compat/__init__.py:180: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  warn(


In [5]:
dataset_name = 'DM_rep4'
dataset_color = 'black'
modality_names = ['RNA', 'ATAC']

# data1 = PCA(n_components=32).fit_transform(data1)
# data2 = PCA(n_components=32).fit_transform(data2)
# fnames1 = fnames2 = None

type1 = np.array(len(data1) * ['Cell Type 0'])
type2 = np.array(len(data2) * ['Cell Type 0'])

# Sampling
# sample_num = 500
# data_col_idx = np.random.choice(range(split), sample_num, replace=False)
# data1, data2, type1, type2 = (x[list(data_col_idx) + list(range(split, len(data1)))] for x in (data1, data2, type1, type2))
# split = sample_num

# Labels
labels = [type1, type2]
features = [np.array(fnames1), np.array(fnames2)]
feature_dict = {}

# Utility
positivize = lambda X: [x + x.min() for x in X]
minmax = lambda X: [(x + x.min()) for x in X]

In [5]:
# Preprocessing
data1 = preprocessing.scale(data1, axis=0)
data2 = preprocessing.scale(data2, axis=0)
data1[np.isnan(data1)] = 0  # Replace NaN with average
data2[np.isnan(data2)] = 0
# data1 = preprocessing.MinMaxScaler().fit_transform(data1)
# data2 = preprocessing.MinMaxScaler().fit_transform(data2)
dataset = [data1, data2]

# Replace NULL feature names
for i in range(len(features)):
    if features[i] is None:
        features[i] = np.array([f'Feature {i}' for i in range(dataset[i].shape[1])])
        
# Train-Test Imputation
train_size = int(.8 * len(data1))
train_idx = np.random.choice(range(len(data1)), train_size, replace=False)
test_idx = np.array(list(set(range(len(data1))) - set(train_idx)))

# Reduced Priors
full_priors = np.eye(len(dataset[0]))
random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)
none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

In [ ]:
reduced_dim = 32

# ComManDo

In [ ]:
# Imputation
kwargs = {
    'output_dim': reduced_dim,
    'epoch_DNN': 10000,
    'min_epochs': 2500,
    'log_DNN': 500,
    'use_early_stop': True,
    'batch_size': 512,
    'pca_dim': 2*[512],
    'dist_method': 'euclidean',
    'loss_weights': [1,1,1,1],
}
size_str, hash_str = hash_kwargs(kwargs, dataset_name, dataset)
prefix = 'saved_models/cm_im---'
model_str = prefix + hash_str + '.h5'
match_str = prefix + size_str + '.npy'

# Instantiate
mr = list(np.load(match_str, allow_pickle=True)) if os.path.exists(match_str) else None
cm_im = ComManDo(**kwargs, match_result=mr, debug=True)

# Train
if os.path.exists(model_str):
    cm_im.load_model(model_str)
    print(f'Loaded model \'{model_str}\'')
else:
    cm_im.fit_transform(dataset=[d[train_idx] for d in dataset])
    cm_im.save_model(model_str)
    np.save(match_str, cm_im.match_result, allow_pickle=True)
cm_imputed = [cm_im.modal_predict(dataset[i], i) for i in range(1, -1, -1)]

# Comparison Methods

In [ ]:
# Imputation
if True:
    imputation_name = 'Babel'
    bdata1 = sc.read_h5ad(babel_dir + 'atac_rna_test_preds.h5ad').X.toarray()
    bdata2 = sc.read_h5ad(babel_dir + 'rna_atac_test_preds.h5ad').X.toarray()
    nn_imputed = [bdata1, bdata2]
else:
    imputation_name = 'KNN'
    nn_imputed = [predict_knn(torch.tensor(dataset[i][train_idx]).float(), torch.tensor(dataset[(i+1)%2][train_idx]).float(), torch.tensor(dataset[i]).float()) for i in range(1, -1, -1)]
imputed_data_full = [cm_imputed, nn_imputed][::-1]
imputed_data = [[d[test_idx] for d in ds] for ds in imputed_data_full]
imputed_names = ['JAMIE', imputation_name][::-1]

# Visualization

In [ ]:
style='white'
sns.set(style=style)
plt.rcParams.update({'font.weight': 'normal',
                     'font.size': 18,
                     'axes.titlesize': 'large',
                     'axes.labelsize': 'large',
                     'xtick.labelsize': 'small',
                     'ytick.labelsize': 'small'})

In [ ]:
plt.figure(figsize=(10, 5))
plot_auroc_correlation(imputed_data, [data1[test_idx], data2[test_idx]], modality_names, index=0, names=imputed_names)
plt.tight_layout()
plt.savefig('./output_figures/' + dataset_name + '-Imp1.png', dpi=300)

In [ ]:
plt.figure(figsize=(10, 5))
plot_auroc_correlation(imputed_data, [data1[test_idx], data2[test_idx]], modality_names, index=1, names=imputed_names)
plt.tight_layout()
plt.savefig('./output_figures/' + dataset_name + '-Imp2.png', dpi=300)

In [ ]:
plt.figure(figsize=(10, 5))
plot_distribution_alone([dataset[0][test_idx], cm_imputed[0]], 2*[labels[0][test_idx]], title=modality_names[0], fnames=2*[features[0]], feature_dict=feature_dict)
sns.despine()
plt.savefig('./output_figures/' + dataset_name + '-Dist1.png', dpi=300)

In [ ]:
plt.figure(figsize=(10, 5))
plot_distribution_alone([dataset[1][test_idx], cm_imputed[1]], 2*[labels[1][test_idx]], title=modality_names[1], fnames=2*[features[1]], feature_dict=feature_dict)
sns.despine()
plt.savefig('./output_figures/' + dataset_name + '-Dist2.png', dpi=300)

# Explanation

In [ ]:
mod0, mod1 = 1, 0
current_cm = cm

In [ ]:
from scipy import stats

# Omit one testing
in_data = dataset[mod0].copy()
target = interest[0]  # Imputation
target_name = features[mod1][target]  # Imputation
# target_name = 'LTA'  # LTA
out_data = dataset[mod1][:, target]  # Imputation

background = in_data.mean(0)
logits = current_cm.modal_predict(in_data, mod0)[:, target]  # Imputation
# logits = current_cm.test_LabelTA(2*[current_cm.transform_one(in_data, mod0)], labels)  # LTA
baseline = stats.pearsonr(logits, out_data)  # Imputation
baseline = logits  # LTA

performance = []
best_idx = -1
best_perf = -np.inf
best_str = ''
check_best = 10
# testing_idx = np.random.choice(dataset[mod0].shape[1], int(1e3), replace=False)
testing_idx = np.array(range(in_data.shape[1]))
for i, idx in enumerate(testing_idx):
    # CLI
    if (i+1) % check_best == 0 and len(performance) > 0:
        best_idx = np.argmax(-np.array(performance))  # Replace one
        # best_idx = np.argmax(performance)  # Keep one
        best_perf = performance[best_idx]
        best_str = features[mod0][testing_idx[best_idx]]
    prog_str = math.floor(50*(i+1)/len(testing_idx)) * '|'
    print(
        f'{i+1:>{len(str(len(testing_idx)))}}/{len(testing_idx)} [{prog_str:<50}] - '
        f'Current Best: {best_perf:.5f}, {best_str}'
        , end='\r')
    
    mod_data = in_data
    # Replace one
    replace_idx = idx  # Replace one
    # replace_idx = [i!=idx for i in range(mod_data.shape[1])]  # Keep one
    old_data = mod_data[:, replace_idx]
    mod_data[:, replace_idx] = background[replace_idx]
    
    # Predict
    # logits = current_cm.modal_predict(mod_data, mod0)[:, target]  # Imputation
    logits = current_cm.test_LabelTA(2*[current_cm.transform_one(mod_data, mod0)], labels)  # LTA
    
    # Repair
    mod_data[:, replace_idx] = old_data
    
    # Record
    # perf = stats.pearsonr(logits, out_data)[0]  # Imputation
    # # perf = -np.sum((logits - out_data)**2)  # Imputation
    perf = logits  # LTA
    if np.isnan(perf):
        perf = -np.inf
    performance.append(perf)
print('\nDone!')
performance = np.array(performance)

In [ ]:
argsort_perf = np.argsort(performance)  # Replace one
# argsort_perf = np.argsort(performance)[::-1]  # Keep one

sorted_idx = testing_idx[argsort_perf]
sorted_names = features[mod0][sorted_idx]
sorted_perf = performance[argsort_perf]

display = 20
print(f'Target {modality_names[mod1]} Feature: {target_name}')
print(f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={target_name}')
importance = features[mod0][sorted_idx][:display]
imp_str = ', '.join(importance)
print(f'Important {modality_names[mod0]} Features: {imp_str}')
for n, p in zip(importance, sorted_perf[:display]):
    print(f'{p:.5f}: https://www.genecards.org/cgi-bin/carddisp.pl?gene={n}')

In [ ]:
file_contents = np.concatenate([sorted_idx.reshape((-1, 1)), sorted_names.reshape((-1, 1)), sorted_perf.reshape((-1, 1))], axis=1)
file_contents_str = '\n'.join([' '.join([str(s) for s in r]) for r in file_contents]) + '\n'
with open(f'output_data/JAMIE---{size_str}---sorted_genes', 'w') as f:
    f.write(file_contents_str)

plt.figure(figsize=(15, 5))
plot_impact(performance, features[mod0], baseline, max_features=30, color=dataset_color)
sns.despine()
plt.tight_layout()
plt.savefig('./output_figures/' + dataset_name + '-Analysis.png', dpi=300)

# SHAP

In [ ]:
# RNA (Place) -> ATAC
# ENSG00000251562 (4) -> HYLS1
# ENSG00000251562 (4) -> HYLS1

# ATAC (Place) -> RNA
# ANKRD16 -> ENSG00000138119
# PWWP2B

In [ ]:
# import shap

# # Subset for runtime
# use_subsetting = True
# subset_idx = np.random.choice(dataset[mod0].shape[1], 100, replace=False)
# background = np.mean(dataset[mod0], axis=0).reshape((1, -1))
# def take_subset(x, do=use_subsetting):
#     if not do:
#         return x
#     if len(x.shape) == 1:
#         return x[subset_idx]
#     return x[:, subset_idx]
# def reverse_subset(x, do=use_subsetting):
#     if not do:
#         return x
#     new_vec = np.concatenate(x.shape[0] * [background], axis=0)
#     new_vec[:, subset_idx] = x
#     return new_vec

# target_feature = interest[0]
# model = lambda x: current_cm.modal_predict(reverse_subset(x), mod0)[:, target_feature]
# data = dataset[mod0]
# back = background
# feature_names = features[mod0]
# output_names = features[mod1][target_feature]

# # Apply subsetting
# data = take_subset(data)
# back = take_subset(back)
# feature_names = list(take_subset(np.array(feature_names)))

# # Explainer
# explainer = shap.Explainer(model, back, feature_names=feature_names, output_names=output_names)
# shap_values = explainer(data, max_evals=2*data.shape[1]+1)

In [ ]:
# display = 20
# print(f'Target {modality_names[mod1]} Feature: {output_names}')
# for n in [output_names]:
#     print(f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={n}')
# importance = np.array(shap_values.feature_names)[np.argsort(shap_values.abs.mean(0).values)][::-1][:display]
# # importance = np.array(shap_values.feature_names)[np.argsort(shap_values.mean(axis=0).abs.values)[::-1][:display]]
# # importance = np.array(shap_values.feature_names)[np.argsort(shap_values.abs.max(axis=0).abs.values)[::-1][:display]]
# imp_str = ', '.join(importance)
# print(f'Important {modality_names[mod0]} Features: {imp_str}')
# for n in importance:
#     print(f'https://www.genecards.org/cgi-bin/carddisp.pl?gene={n}')

In [ ]:
# plt.figure(figsize=(10, 5))
# ax = plt.gcf().add_subplot(1, 1, 1)
# shap.summary_plot(shap_values, data, plot_type='violin', max_display=display, plot_size=(9.2, 5), show=False)
# plt.tight_layout()
# plt.savefig('./output_figures/' + dataset_name + '-Summary.png');

In [ ]:
# plt.figure(figsize=(10, 5))
# shap.plots.heatmap(shap_values, max_display=7, show=False)  # 7 because problem with visualization and tight layout
# plt.gca().set_xlabel('Samples')
# plt.tight_layout()
# plt.savefig('./output_figures/' + dataset_name + '-Heat.png');

In [ ]:
# plt.figure(figsize=(10, 5))
# ax = plt.gcf().add_subplot(1, 1, 1)
# shap.dependence_plot(np.argsort(np.abs(shap_values.values).mean(0))[::-1][0], shap_values.values, data, ax=ax, show=False)
# plt.tight_layout()
# plt.savefig('./output_figures/' + dataset_name + '-Dep1.png');

In [ ]:
# plt.figure(figsize=(10, 5))
# ax = plt.gcf().add_subplot(1, 1, 1)
# shap.plots.waterfall(100*shap_values[0], show=False)
# plt.tight_layout()
# plt.savefig('./output_figures/' + dataset_name + '-Waterfall1.png');

## Interactive

In [ ]:
# # Kernel Explainer
# kexplainer = shap.KernelExplainer(model, shap.kmeans(data, 10), feature_names=feature_names, output_names=output_names)
# import warnings
# with warnings.catch_warnings():
#     warnings.simplefilter('ignore')
#     kshap_values = kexplainer.shap_values(data, nsamples=100)

In [ ]:
# shap.force_plot(kexplainer.expected_value, kshap_values[0, :], data[0, :], matplotlib=True, show=False)
# plt.savefig('./output_figures/' + dataset_name + '-Force.png', bbox_inches='tight');

In [ ]:
# shap.initjs()
# shap.force_plot(kexplainer.expected_value, kshap_values, data)